In [6]:
import arxiv
import os
import tarfile
import preprocess_paper as preprocessor
import time

# Construct the default API client.
client = arxiv.Client()

def search_cat(cat, tmpdir, dirpath, required_papers=10):
    search = arxiv.Search(
    query = f"cat:{cat}",
    sort_by = arxiv.SortCriterion.SubmittedDate
    )

    successful_downloads = 0

    for i,r in enumerate(client.results(search)):
        filename = f"paper_{i}.tar.gz"
        time.sleep(2)
        try:
            filepath = r.download_source(dirpath=tmpdir, filename=filename)
        except Exception as e:
            print(f"Error downloading Paper {i}: {e}")
            continue
        successful_extract = False
        tex_path = os.path.join(dirpath, f'{cat.split(".")[1]}_paper_{i}.tex')
        try:
            with tarfile.open(filepath, 'r:gz') as tar:
                files = tar.getmembers()
                tex_files = [f.name for f in files if f.name.endswith('.tex')]
                if len(tex_files) == 1:
                    tex_file = tex_files[0]
                    print(f"Extracting {tex_file}")
                    tar.extract(tex_file, path=dirpath)
                    os.rename(os.path.join(dirpath, tex_file), tex_path)
                    successful_extract = True
                else:
                    print(f"Paper {i} has {len(tex_files)} tex files. Skipping.")
                tar.close()
        except Exception as e:
            print(f"Error extracting {filepath}: {e}")
        os.remove(filepath)

        if successful_extract:
            try:
                successful_pp = preprocessor.preprocess(tex_path)
            except Exception as e:
                successful_pp = False
            
            if successful_pp:
                successful_downloads += 1
            else:
                print(f"Error preprocessing {tex_path}. Skipping.")
                os.remove(tex_path)

        if successful_downloads == required_papers:
            break

    return successful_downloads


# Arxiv categories
cats = {
    # 'Computer Science': ['cs.' + cat for cat in [
    #     'AI', 'AR', 'CC', 'CE', 'CG', 'CL', 'CR', 'CV', 'CY', 'DB',
    #     'DC', 'DL', 'DM', 'DS', 'ET', 'FL', 'GL', 'GR', 'GT', 'HC',
    #     'IR', 'IT', 'LG', 'LO', 'MA', 'MM', 'MS', 'NA', 'NE', 'NI',
    #     'OH', 'OS', 'PF', 'PL', 'RO', 'SC', 'SD', 'SE', 'SI', 'SY',
    # ]],
    # 'Mathematics': ['math.' + cat for cat in [
    #     'AT', 'CA', 'CO', 'CT', 'CV', 'DG', 'DS', 'FA',
    #     'GM', 'GN', 'GR', 'GT', 'HO', 'IT', 'KT', 'LO', 'MG', 'MP',
    #     'NA', 'NT', 'OA', 'OC', 'PR', 'QA', 'RT', 'RA', 'SP', 'ST',
    # ]],
    # 'Quantitative Finance': ['q-fin.' + cat for cat in [
    #     'CP', 'EC', 'GN', 'MF', 'PM', 'PR', 'RM', 'ST', 'TR',
    # ]],
    # 'Statistics': ['stat.' + cat for cat in [
    #     'AP', 'CO', 'ME', 'ML', 'OT', 'TH',
    # ]],
    'Economics': ['econ.' + cat for cat in ['TH']],
}
tmpdir = './tarfiles'

if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

for cat, subcats in cats.items():
    dirpath = f'./selected_papers/{cat}'
    required_papers = 1

    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
    print(f"Searching for papers in category {cat}")
    for subcat in subcats:
        print(f"Searching for papers in subcategory {subcat}")
        required_papers -= search_cat(subcat, tmpdir, dirpath, required_papers=required_papers)
        if required_papers == 0:
            break
    print("--------------------------------------------------------------")
    time.sleep(5)

os.removedirs(tmpdir)


Searching for papers in category Economics
Searching for papers in subcategory econ.GN
Extracting ecai_submission.tex
Preprocessing GN_paper_0.tex: 
Error preprocessing ./selected_papers/Economics/GN_paper_0.tex. Skipping.
Error extracting ./tarfiles/paper_1.tar.gz: not a gzip file
Extracting ms.tex
Preprocessing GN_paper_2.tex: 
Preprocessed text is too long: 36561 tokens
Error preprocessing ./selected_papers/Economics/GN_paper_2.tex. Skipping.
Error extracting ./tarfiles/paper_3.tar.gz: not a gzip file
Extracting Manuscript.tex
Preprocessing GN_paper_4.tex: 
Error preprocessing ./selected_papers/Economics/GN_paper_4.tex. Skipping.
Paper 5 has 3 tex files. Skipping.
Paper 6 has 3 tex files. Skipping.
Extracting main.tex
Preprocessing GN_paper_7.tex: 
Error preprocessing ./selected_papers/Economics/GN_paper_7.tex. Skipping.
Paper 8 has 7 tex files. Skipping.
Paper 9 has 3 tex files. Skipping.
Paper 10 has 14 tex files. Skipping.
Error extracting ./tarfiles/paper_11.tar.gz: not a gzip f

KeyboardInterrupt: 